In [163]:
import pandas as pd
import numpy as np
import altair as alt

# Comparing the winning rate of different nation 

in All men's doubles (MD) matches played during the BWF World Tour.

## 1. merge data

In [164]:
file_name = ["md"]

final_data = pd.DataFrame()
for f in file_name:
    d = pd.read_csv("data/" + f + ".csv")
    if "team_one_nationalities" not in d.columns:
        d["team_one_nationalities"] = d["team_one_player_one_nationality"]
        d["team_two_nationalities"] = d["team_two_player_one_nationality"]
    final_data = pd.concat([final_data, d])

In [165]:
final_data.to_csv("data/final_data.csv")

## 2. operate data

In [166]:
data = pd.read_csv("data/final_data.csv")
data.sample(5)

,Unnamed: 0,tournament,city,country,date,tournament_type,discipline,round,winner,nb_sets,...,team_one_game_points_game_2,team_two_game_points_game_2,game_2_scores,team_one_most_consecutive_points_game_3,team_two_most_consecutive_points_game_3,team_one_game_points_game_3,team_two_game_points_game_3,game_3_scores,team_one_nationalities,team_two_nationalities
427,427,2018 YONEX Canada Open,Calgary,Canada,24-06-2018,BWF Tour Super 100,MD,Final,1,3,...,2.0,0.0,"['0-0', '0-1', '0-2', '1-2', '1-3', '1-4', '2-...",5.0,3.0,1.0,3.0,"['0-0', '1-0', '1-1', '1-2', '1-3', '2-3', '2-...",ENG,GER
1433,1433,Lingshui China Masters 2019,Lingshui,China,13-03-2019,BWF Tour Super 100,MD,Round of 32,2,3,...,0.0,2.0,"['0-0', '1-0', '2-0', '2-1', '3-1', '3-2', '3-...",3.0,6.0,0.0,2.0,"['0-0', '1-0', '1-1', '2-1', '2-2', '2-3', '2-...",KOR,CHN
353,353,CROWN GROUP Australian Open 2018,Sydney,Australia,11-05-2018,HSBC BWF World Tour Super 300,MD,Quarter final,1,2,...,1.0,0.0,"['0-0', '0-1', '0-2', '0-3', '1-3', '2-3', '2-...",NaN,NaN,NaN,NaN,[],INA,JPN
2119,2119,YUZU Indonesia Masters 2019,Malang,Indonesia,04-10-2019,BWF Tour Super 100,MD,Quarter final,2,2,...,0.0,1.0,"['0-0', '1-0', '1-1', '1-2', '2-2', '3-2', '3-...",NaN,NaN,NaN,NaN,[],JPN,CHN
699,699,DAIHATSU YONEX Japan Open 2018,Tokyo,Japan,12-09-2018,HSBC BWF World Tour Super 750,MD,Round of 32,2,2,...,0.0,1.0,"['0-0', '1-0', '2-0', '2-1', '3-1', '3-2', '4-...",NaN,NaN,NaN,NaN,[],IND,JPN


In [167]:
len(data)

2804

In [168]:
diff_nat_data = data[data['team_one_nationalities'] != data["team_two_nationalities"]]
winner_nat = []
loser_nat = []
for i in range(len(diff_nat_data)):
    if data.iloc[i]['winner'] == 1:
        winner_nat.append(data.iloc[i]["team_one_nationalities"])
        loser_nat.append(data.iloc[i]["team_two_nationalities"])
    else:
        winner_nat.append(data.iloc[i]["team_two_nationalities"])
        loser_nat.append(data.iloc[i]["team_one_nationalities"])
diff_nat_data['winner_nationalities'] = winner_nat
diff_nat_data['loser_nationalities'] = loser_nat

/var/folders/v2/p8xqdfwn73b7h929lqr4xpjm0000gn/T/ipykernel_39273/2529909722.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_nat_data['winner_nationalities'] = winner_nat
/var/folders/v2/p8xqdfwn73b7h929lqr4xpjm0000gn/T/ipykernel_39273/2529909722.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_nat_data['loser_nationalities'] = loser_nat


### 2.1 get nation data

In [169]:
nation_data = pd.DataFrame()
nat_winning_rate = {}
for i in range(len(diff_nat_data)):
    cur_winning_nat = diff_nat_data.iloc[i]['winner_nationalities']
    cur_losing_nat = diff_nat_data.iloc[i]['loser_nationalities']
    if cur_winning_nat not in nat_winning_rate.keys():
        nat_winning_rate[cur_winning_nat] = [1, 0]
    else:
        nat_winning_rate[cur_winning_nat][0] += 1
    cur_data = pd.DataFrame({'nation':[cur_winning_nat],'date': [diff_nat_data.iloc[i]['date']], 'winning_matches':[nat_winning_rate[cur_winning_nat][0]], 'losing_matches':[nat_winning_rate[cur_winning_nat][1]]})
    nation_data = pd.concat([nation_data,cur_data])
    
    if cur_losing_nat not in nat_winning_rate.keys():
        nat_winning_rate[cur_losing_nat] = [0, 1]
    else:
        nat_winning_rate[cur_losing_nat][1] += 1 
    cur_data = pd.DataFrame({'nation':[cur_losing_nat],'date': [diff_nat_data.iloc[i]['date']], 'winning_matches':[nat_winning_rate[cur_losing_nat][0]], 'losing_matches':[nat_winning_rate[cur_losing_nat][1]]})
    nation_data = pd.concat([nation_data,cur_data])

In [170]:
nation_data = nation_data.reset_index().drop(["index"], axis = 1)
nation_data["total_matches"] = nation_data["winning_matches"] + nation_data["losing_matches"]
nation_data["winning_rate"] = nation_data["winning_matches"] / nation_data["total_matches"]
nation_data["final_day"] = np.zeros(len(nation_data))

In [171]:
nat = []
nat_total_cnt = len(nation_data["nation"].unique())
for i in range(len(nation_data) - 1, 0, -1):
    if len(nat) == nat_total_cnt:
        break
    if nation_data.iloc[i]["nation"] not in nat:
        nat.append(nation_data.iloc[i]["nation"])
        nation_data.loc[i, "final_day"] = 1

In [172]:
nation_data['date'] = pd.to_datetime(nation_data['date'])

/var/folders/v2/p8xqdfwn73b7h929lqr4xpjm0000gn/T/ipykernel_39273/1014992418.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  nation_data['date'] = pd.to_datetime(nation_data['date'])


In [173]:
nation_data = nation_data.groupby(['nation', 'date']).median().reset_index().sort_values(by = ['nation', 'date'])
nation_data['winning_rate'] = nation_data['winning_matches'] / nation_data['total_matches']
nation_data

,nation,date,winning_matches,losing_matches,total_matches,winning_rate,final_day
0,ALG,2018-11-04,0.0,1.0,1.0,0.000000,0.0
1,ALG,2019-03-20,0.0,2.0,2.0,0.000000,0.0
2,ALG,2019-04-07,0.0,3.0,3.0,0.000000,1.0
3,AUS,2018-01-17,1.0,2.0,3.0,0.333333,0.0
4,AUS,2018-05-09,3.0,14.0,17.0,0.176471,0.0
...,...,...,...,...,...,...,...
2418,VIE,2019-02-28,1.0,12.5,13.5,0.074074,0.0
2419,VIE,2019-05-09,2.0,14.0,16.0,0.125000,0.0
2420,VIE,2019-06-09,2.0,15.0,17.0,0.117647,0.0
2421,VIE,2019-10-30,2.5,16.0,18.5,0.135135,0.0


## 3. plot

In [181]:
#change color when selection
selection=alt.selection_interval(encodings = ["x"],empty="none")

colorCondition=alt.condition(selection, "nation", alt.value("lightgray"))

#scatter plot
scatter_chart = alt.Chart(nation_data).mark_circle(filled = True, size = 60).transform_filter(
    alt.datum.final_day == 1
).encode(
    x=alt.X('total_matches:Q', title = "final total matches"),
    y=alt.Y('winning_rate:Q', title = "Winning rate of the nation"),
    color = alt.value("lightgray"),
    tooltip=alt.Text(
        ['nation', 'winning_rate:Q', 'total_matches:Q']
    )
)

# line chart
line_chart = alt.Chart(nation_data).mark_line(interpolate='step', size=2).encode(
    alt.Y('winning_rate:Q', title = "Winning rate of the nation", impute=alt.ImputeParams(value=None)),
    alt.X('yearmonth(date):T'),
    alt.Color('nation:N'),
    tooltip=['nation', 'winning_rate', 'total_matches'],
).properties(
    width=600,
    height=300,
).interactive()

# add selection to each component
scatter_chart = scatter_chart.add_selection(
    selection
).encode(
    color=colorCondition,
).properties(
    width=600,
    height=300,
)

line_chart = line_chart.transform_filter(selection)


#Put all together
(scatter_chart & line_chart).configure_view(strokeWidth=0).properties(
    title = "The winning rate of different nation in All men's doubles (MD) matches played during the BWF World Tour."
).configure_point(
    size=5
)



alt.VConcatChart(...)